<a href="https://colab.research.google.com/github/amien1410/amien-scrapers/blob/main/Crexi_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
!pip install xlsxwriter

import requests
import time
import json
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import files
from itertools import cycle

# get state polygons
!wget 'https://github.com/amien1410/scraping-projects/raw/main/sp-big-v3.json'
!wget 'https://raw.githubusercontent.com/TheSpeedX/PROXY-List/master/socks4.txt'
with open('/content/sp-big-v3.json', "rb") as f:
   json_data = json.load(f)

# # opening the CSV file
proxies = []
with open('/content/socks4.txt', mode ='r') as f:
   lines = f.readlines()
   for l in lines:
         proxies.append(l.strip())

def get_polygons(state):
  for item in json_data:
    if item['state'] == state:
      my_item = item
      break
  else:
    return False
  return my_item['polygons']

def get_properties(name, search_type, filter):
  # payload = {"locations":[{"placeId":"ChIJSx6SrQ9T2YARed8V_f0hOg0","type":"city","city":city,"polygons":[{"coordinates":[]},{"coordinates":[]},{"coordinates":[]},{"coordinates":[]}]}],"count":1000,"mlScenario":"search-properties","offset":0,"userId":"$device:1892ba1bdda2059-09edcca6e4277e-7e56547f-13c680-1892ba1bdda2059","sortDirection":"Descending","sortOrder":"rank","includeUnpriced":True}
  results = []
  hasNextPage = True
  offset = 0
  prox = cycle(proxies)

  while hasNextPage == True:
    proxy = next(prox)
    if search_type == "State":
      polygons = get_polygons(name)
      if polygons == False:
        print("wrong state!")
        return False
      payload = {
        "locations": [
          {
            "placeId": "ChIJdf5LHzR_hogR6czIUzU0VV4",
            "type": "stateCode",
            # "stateCode": "AL",
            "polygons": polygons
          }
        ],
        "types": [filter],
        "count": 500,
        "mlScenario": "search-properties",
        "offset": offset,
        "excludeAssetIds": [],
        "userId": "$device:1892b8ff0074068-088509215b1abe-7e56547f-100200-1892b8ff0074068",
        "sortDirection": "Descending",
        "sortOrder": "rank",
        "includeUnpriced": True
      }
    else:
      payload = {
        "locations": [
          {
            "placeId": "ChIJSx6SrQ9T2YARed8V_f0hOg0",
            "type": "city",
            "city": name,
            "polygons": [
              {
                "coordinates": []
              },
              {
                "coordinates": []
              },
              {
                "coordinates": []
              },
              {
                "coordinates": []
              }
            ]
          }
        ],
        "types": [filter],
        "count": 500,
        "mlScenario": "search-properties",
        "offset": offset,
        "userId": "$device:1892ba1bdda2059-09edcca6e4277e-7e56547f-13c680-1892ba1bdda2059",
        "sortDirection": "Descending",
        "sortOrder": "rank",
        "includeUnpriced": True
      }
    url = "https://api.crexi.com/assets/search"
    r = requests.post(url, json=payload, proxies={'http':'http://'+proxy})
    data = r.json()

    if 'errors' in data:
      print("Properties can be shown for you just 1500 properties")
      return results

    for i in range(len(data['data'])):
      try:
        results.append({
            "propertyId": data['data'][i]['id'],
            "propertyName": data['data'][i]['name'],
            "url": data['data'][i]['urlSlug']
        })

      except:
        continue

    if len(data['data']) < 500:
      hasNextPage = False

    offset += 500
    time.sleep(2)

  print("Found: " + str(len(results)) + " properties.")
  return results


async def get_property_info(propertyId):

    url = "https://api.crexi.com/assets/" + str(propertyId)
    print(f"Property URL: {url}")
    r = requests.get(url)
    data = r.json()
    # print(data)
    details = {k.replace(' ', ''): v for k, v in data["details"].items()}
    # print(details)

    try:
      highlights = data.get('investmentHighlights',"")
      cleanHighlights = BeautifulSoup(highlights, "html.parser").text
      marketingDescription = data.get('marketingDescription',"")
      cleanMarketingDescription = BeautifulSoup(marketingDescription, "html.parser").text
      askingPrice = data.get("askingPrice", "")
      property_type = data["details"].get("Property Type", "")
      subtype = data["details"].get("Subtype", "")
      investment_type = data["details"].get("Investment Type", "")
      leaseType = details.get("LeaseType", "")
      tenantCredit = details.get("TenantCredit", "")
      tenancy = details.get("Tenancy", "")
      brandt = data["details"].get("Brand/Tenant", "")
      lease_tearm = data["details"].get("Lease Term", "")
      lease_commencement = data["details"].get("Lease Commencement", "")
      lease_expiration = data["details"].get("Lease Expiration", "")
      remaining_term = data["details"].get("Remaining Term", "")
      address = data["locations"][0].get("address", "")
      city = data["locations"][0].get("city", "")
      state = data["locations"][0].get("state", {}).get("code", "")
      zip_code = data["locations"][0].get("zip", "")
      latitude = data["locations"][0].get("latitude", "")
      longitude = data["locations"][0].get("longitude", "")
      square_footage = data["details"].get("Square Footage", "")
      price_sq_ft = data["details"].get("Price/Sq Ft", "")
      cap_rate = data["details"].get("Cap Rate", "")
      occupancy = data["details"].get("Occupancy", "")
      noi = data["details"].get("NOI", "")
      year_built = data["details"].get("Year Built", "")
      permitted_zoning = data["details"].get("Permitted Zoning", "")
      rent_bumps = data["details"].get("Rent Bumps", "")
      apn = data["details"].get("APN", "")
      lease_options = data["details"].get("Lease Options", "")
      fee_agreement_required = data.get("feeAgreementIsRequired", "")
      user_can_download_md = data.get("userCanDownloadMd", "")
      user_executed_ca = data.get("userExecutedCA", "")
      is_saved_to_boards = data.get("isSavedToBoards", "")
      ca_principal_info_type = data.get("caPrincipalInfoType", "")
      is_in_opportunity_zone = data.get("isInOpportunityZone", "")
      is_note_loan = data.get("isNoteLoan", "")
      deposit_option = data.get("depositOption", "")
      broker_plan = data.get("brokerPlan", "")
      is_sold = data.get("isSold", "")
      is_paused = data.get("isPaused", "")
      is_outdated = data.get("isOutdated", "")
      status = data.get("status", "")
      id = data.get("id", "")
      name = data.get("name", "")
      description = data.get("description", "")
      thumbnail_url = data.get("thumbnailUrl", "")
      is_private = data.get("isPrivate", "")
      url_property = "https://www.crexi.com/properties/" + str(id) + "/" + data.get("urlSlug", "")
      is_comp = data.get("isComp", "")
      is_elite = data.get("isElite", "")
      is_sell_subject_to = data.get("isSellSubjectTo", "")

      return {
        'isPaid': data["isPaid"],
        'investmentHighlights': cleanHighlights,
        'marketingDescription': cleanMarketingDescription,
        'Asking Price': askingPrice,
        'Property Type': property_type,
        'Subtype': subtype,
        'Investment Type': investment_type,
        'Lease Type': leaseType,
        'Tenant Credit': tenantCredit,
        'Tenancy': tenancy,
        'Brand/Tenant': brandt,
        'Lease Term': lease_tearm,
        'Lease Commencement': lease_commencement,
        'Lease Expiration': lease_expiration,
        'Remaining Term': remaining_term,
        'Square Footage': square_footage,
        'Price/Sq Ft': price_sq_ft,
        'Cap Rate': cap_rate,
        'Occupancy': occupancy,
        'NOI': noi,
        'Year Built': year_built,
        'Permitted Zoning': permitted_zoning,
        'Rent Bumps': rent_bumps,
        'APN': apn,
        'Lease Options': lease_options,
        'createdOn': data["createdOn"],
        'activatedOn': data["activatedOn"],
        'Address': address,
        'City': city,
        'State': state,
        'Zip Code': zip_code,
        'Latitude': latitude,
        'Longitude': longitude,
        'Fee Agreement Required': fee_agreement_required,
        'User Can Download MD': user_can_download_md,
        'User Executed CA': user_executed_ca,
        'Is Saved to Boards': is_saved_to_boards,
        'CA Principal Info Type': ca_principal_info_type,
        'Is in Opportunity Zone': is_in_opportunity_zone,
        'Is Note Loan': is_note_loan,
        'Deposit Option': deposit_option,
        'Broker Plan': broker_plan,
        'Is Sold': is_sold,
        'Is Paused': is_paused,
        'Is Outdated': is_outdated,
        'Status': status,
        'ID': id,
        'Name': name,
        'Description': description,
        'Thumbnail URL': thumbnail_url,
        'Is Private': is_private,
        'URL Property': url_property,
        'Is Comp': is_comp,
        'Is Elite': is_elite,
        'Is Sell Subject To': is_sell_subject_to
    }
    except:
      return None


async def main(name, type, filter):
  results = []
  properties = get_properties(name, type, filter)
  for i in range(len(properties)):
    print(f"Task: {i}")
    try:
      print(f"Property ID: {properties[i]['propertyId']}")
      data = await get_property_info(properties[i]["propertyId"])
      results.append(data)
    except:
      continue
  results = [x for x in results if x is not None]
  df = pd.DataFrame(results)
  df.to_excel("Florida-Crexi.xlsx", engine='xlsxwriter')
  files.download("Florida-Crexi.xlsx")
  return df

In [ ]:
await main("Florida", "State", "Multifamily")
# await main("San Diego", "City", "Retail")
# get_properties("Arizona", "State", "Retail")